In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
plt.style.use('fivethirtyeight')

sales.csv   - Данные по продажам за ноябрь 2015 г. в детализации День – Магазин – Товар - Сумма_Продаж

txn.csv     - Данные по продажам за период [2015-10-01, 2015-11-29] в детализации День – Магазин – Товар – Чек - Сумма_Продаж.

art.csv     - Справочник товаров

store.csv   - Справочник магазинов

In [2]:
art = pd.read_csv('art.csv', sep=';')
sales = pd.read_csv('sales.csv', sep=';')
store = pd.read_csv('store.csv', sep=';')
txn = pd.read_csv('txn.csv', sep=';')

In [3]:
#Преобразование строки c ценой в вещественное значение
txn['SALES'] = txn['SALES'].apply(lambda x: float(x.replace(',','.')))

In [4]:
#добавляем информацию по товару и товарной группе
txn = pd.merge(txn, art, how='left', on='ART')

In [5]:
txn.head()

,DAY,STORE,ART,TXN,TIME,SALES,NAME,GRP
0,2015-11-05,22691,16548710,5877969776,17:23:57,117.00,cигареты_4,Сигареты
1,2015-10-24,20581,261204,5767364168,19:57:19,140.17,Курица_7,Мясо курицы охл.
2,2015-11-05,1574,16405210,5876151937,15:31:35,26.90,Крекер_11,Крекеры и галеты
3,2015-10-29,1602,192539,5810625759,15:54:34,53.80,бульон в кубиках_1,Первые блюда
4,2015-10-04,1579,16501864,5575346025,16:27:40,36.30,Нектар_80,Соки


# Задание 2.

### АВС-анализ. Требуется определить минимальный набор товаров (итого по всем магазинам в рамках каждой группы), дающий не менее 50% от объема продаж (категория А) и 80% от объема продаж (А+В). Представить результат в виде таблиц.

Определим функцию для формирования минимального набора товаров для каждого магазина в рамках каждой группы (ABC анализ), где первм шагом выбираем по 1 самомум прибыльному товару в каждой группе, если коммулятивная сумма выручки по выбранным товарам меньше заданнго порога, то продолжаем отбор, но уже концентрируемся только на том, скольо прибыли он приносит. Фунция фомирует таблицу с минимальным набором параметров для 1 магазина.

In [6]:
def get_abc_table(df, lim):
    volume = pd.DataFrame()
    
    #цикл, формирующий таблицу по 1 магазину
    for i in list(df.STORE.unique()):
        df_store = df[df['STORE']==i]
        df_store['id'] = 1

        #сколько выручки дает каждый товар суммированно в каждой группе, отсортировано по убыванию
        df_store = df_store.groupby(['STORE','GRP','ART'])['SALES'].agg('sum').\
                                reset_index().sort_values(by='SALES', ascending=False)
        df_store['%'] = df_store['SALES']/df_store['SALES'].sum()*100
        df_store = df_store.reset_index()
        
        #выбираем max дающие прибыль товары по 1 внутри каждой группы
        df_store_1 = df_store.groupby(['STORE','GRP'])[['index','ART','SALES','%']]\
                                .max().reset_index()
        
        #подсчитываем прибыль от минимального выбранного набора товаров по min 250 группам
        sum_proc = df_store_1['%'].sum()
        
        if (sum_proc < lim):
            id_art = list(df_store_1['index'].unique())
            
            #удаляем уже выбранные товары
            df_store = df_store.loc[~df_store['index'].isin(id_art)]
            df_store = df_store.sort_values(by='%', ascending=False)
            
            #подсчитываем коммулятивную сумму
            df_store['com_%'] = df_store['%'].cumsum()
            df_store = df_store.reset_index(drop=True)
            
            #выбираем товары, которые согласно комм. сумме будут иметь 
            #первое минимальное соответсвие пределу lim (ABC)
            idx = 0
            for i in range(len(df_store)):
                if (df_store['com_%'][i] >= (lim - sum_proc)):
                    idx = i
                    break
                    
            if (idx > 0):
                df_store_2 = df_store.loc[:idx]
                list_col = ['STORE', 'GRP', 'ART', 'SALES', '%']
                end =  pd.concat([df_store_1[list_col],df_store_2[list_col]])
                volume = volume.append(end)
            else:
                volume = volume.append(df_store_1[list_col])                
        else:
            volume =  volume.append(df_count_store_1[list_col])
    return volume.reset_index(drop=True)

### Таблица с минимальным набором товаров, дающий не менее 50% от объема продаж

In [7]:
ABC_table_50 = get_abc_table(txn,50)
ABC_table_50.head()

,STORE,GRP,ART,SALES,%
0,22691,Автотовары,16336697,379.8000,0.004067
1,22691,Апельсины,11849,21203.7072,0.227028
2,22691,БАДы,358587,51.0000,0.000546
3,22691,Байховый чай,16544142,2394.2000,0.025635
4,22691,Бальзамы,368842,760.5000,0.008143


### Таблица с минимальным набором товаров, дающий не менее 80% от объема продаж

In [8]:
ABC_table_80 = get_abc_table(txn, 80)

In [9]:
ABC_table_50

,STORE,GRP,ART,SALES,%
0,22691,Автотовары,16336697,379.8000,0.004067
1,22691,Апельсины,11849,21203.7072,0.227028
2,22691,БАДы,358587,51.0000,0.000546
3,22691,Байховый чай,16544142,2394.2000,0.025635
4,22691,Бальзамы,368842,760.5000,0.008143
5,22691,Бараночные и сухарные изделия,16511277,2772.0000,0.029680
6,22691,Батончики,16547182,7205.2000,0.077146
7,22691,Бифштексы и котлеты,16525118,4779.2000,0.051171
8,22691,Блины,16407980,7850.0000,0.084050
9,22691,Бобовая крупа,16363821,1770.8000,0.018960


In [10]:
ABC_table_80

,STORE,GRP,ART,SALES,%
0,22691,Автотовары,16336697,379.8000,0.004067
1,22691,Апельсины,11849,21203.7072,0.227028
2,22691,БАДы,358587,51.0000,0.000546
3,22691,Байховый чай,16544142,2394.2000,0.025635
4,22691,Бальзамы,368842,760.5000,0.008143
5,22691,Бараночные и сухарные изделия,16511277,2772.0000,0.029680
6,22691,Батончики,16547182,7205.2000,0.077146
7,22691,Бифштексы и котлеты,16525118,4779.2000,0.051171
8,22691,Блины,16407980,7850.0000,0.084050
9,22691,Бобовая крупа,16363821,1770.8000,0.018960
